# 基准方法：基于序列比对的反应推荐
> zhenkun.shi@tib.cas.cn  
> 2024-07-04

Basline methods by using sequence aligmnet method.   

## 1. Import Packages

In [4]:
import sys,os

from sympy import im
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../')
import pandas as pd
from config import conf as cfg
import tools.bioFunctionLib as bfl
from tools import commonfunction as cmfunc
from tkinter import _flatten # type: ignore
from pandarallel import pandarallel 
pandarallel.initialize() 

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def caculate_reaction_pred_true(reaction_groundtruth, reaction_pred):
    set_ground = set(reaction_groundtruth.split(';'))
    set_pred = set(reaction_pred.split(';'))

    if set_ground == set_pred:
        return True
    else:
        return False

## 2. Load expdata

In [3]:
ds_train = pd.read_feather(cfg.FILE_DS_TRAIN)
ds_test = pd.read_feather(cfg.FILE_DS_TEST)

train = ds_train[['uniprot_id','reaction_id','seq',]].rename(columns={'reaction_id':'reaction_groundtruth'})
test = ds_test[['uniprot_id','reaction_id','seq',]].rename(columns={'reaction_id':'reaction_groundtruth'})

#add reaction data
ds_rhea = pd.read_feather(cfg.FILE_DS_RHEA_REACTIONS)

# caculate reaction_id ec_number map
ec_reaction_map = ds_rhea[['reaction_id', 'ec_number']].copy()
ec_reaction_map = ec_reaction_map.fillna('-')
ec_reaction_map.ec_number = ec_reaction_map.ec_number.apply(lambda x: x.replace('EC:',''))
ec_reaction_map = ec_reaction_map.assign(ec_number=ec_reaction_map['ec_number'].str.split(';')).explode('ec_number').reset_index(drop=True).rename(columns={'ec_number': 'ec'})
ec_reaction_map = ec_reaction_map[ec_reaction_map.ec!='-'].reset_index(drop=True)

ds_train.head(3)

,uniprot_id,seq,reaction_id,ec_number,functionCounts,ec_specific_level,isenzyme,label
0,Q6GZX4,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...,-,-,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Q6GZX3,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...,-,-,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Q197F8,MASNTVSAQGGSNRPVRDFSNIQDVAQFLLFDPIWNEQPGSIVPWK...,-,-,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## 3. Blast - Direct

In [4]:
blast_res = bfl.getblast(train=train[['uniprot_id', 'seq']], test=test[['uniprot_id', 'seq']], k=1)         # 序列比对
blast_res = blast_res.merge(train, left_on='sseqid', right_on='uniprot_id', how='left')[['id', 'reaction_groundtruth']].rename(columns={'id':'uniprot_id', 'reaction_groundtruth':'reaction_blast'})
blast_res = test[['uniprot_id', 'reaction_groundtruth']].merge(blast_res, on='uniprot_id', how='left').fillna('NO-PREDICTION')
blast_res.head(3)

,uniprot_id,reaction_groundtruth,reaction_blast
0,A9JLI2,-,-
1,A9JLI3,-,-
2,A9JLI5,-,-


## 4. 保存结果文件

In [5]:
blast_res.to_csv(cfg.FILE_RESULTS_BLAST_DIRECT, sep='\t', index=False)